In [ ]:
data_path = r"D:\Stash\Datasets\db2_janelado\db2_5g_all_data.npz"

data = np.load(data_path)
X_train = data["X_train"]
y_train = data["y_train"]
X_test = data["X_test"]
y_test = data["y_test"]

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

print("Formato final dos dados:", X_train.shape)

Formato final dos dados: (71611, 500, 12)


In [8]:
import numpy as np
def calculate_mpp_mzp(X):
    """
    Calcula MPP e MZP para cada amostra no dataset X.
    Entrada:
        X: np.array de shape (samples, time_steps, channels)
    Retorno:
        np.array de shape (samples, 2) → MPP e MZP por amostra
    """
    mpp = []
    mzp = []

    for sample in X:
        # µ0: energia do sinal (por canal)
        mu0 = np.sqrt(np.sum(sample ** 2, axis=0))  # shape: (channels,)

        # µ2: energia da 1ª derivada (por canal)
        delta = np.diff(sample, axis=0)
        mu2 = np.sqrt(np.sum(delta ** 2, axis=0))  # shape: (channels,)

        # µ4: energia da 2ª derivada (por canal)
        delta2 = np.diff(delta, axis=0)
        mu4 = np.sqrt(np.sum(delta2 ** 2, axis=0))  # shape: (channels,)

        # Ψ = µ4 / µ2, Φ = µ2 / µ0
        psi = np.nan_to_num(mu4 / mu2, nan=0.0, posinf=0.0, neginf=0.0)
        phi = np.nan_to_num(mu2 / mu0, nan=0.0, posinf=0.0, neginf=0.0)

        # MPP = µ0 * Ψ, MZP = µ0 * Φ
        mpp_sample = mu0 * psi
        mzp_sample = mu0 * phi

        # Concatena os dois vetores em um só (por amostra)
        mpp_mzp_concat = np.concatenate([mpp_sample, mzp_sample])  # shape: (channels * 2,)
        mpp.append(mpp_sample)
        mzp.append(mzp_sample)

    # Concatena tudo (samples, features)
    return np.stack(mpp), np.stack(mzp)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Activation, BatchNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt


db2_5g = r"D:\Stash\Datasets\db2_janelado\db2_5g_all_data.npz"
db2_10g = r"D:\Stash\Datasets\db2_janelado\db2_10g_all_data.npz"

db3_5g = r"D:\Stash\Datasets\db3_janelado\db3_5g_all_data.npz"
db3_10g = r"D:\Stash\Datasets\db3_janelado\db3_10g_all_data.npz"

db5_5g = r"D:\Stash\Datasets\db5_janelado\db5_5g_all_data.npz"
db5_10g = r"D:\Stash\Datasets\db5_janelado\db5_10g_all_data.npz"


data = np.load(data_path)
X_train = data["X_train"]
y_train = data["y_train"]
X_test = data["X_test"]
y_test = data["y_test"]


encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

print("Formato final dos dados:", X_train.shape)

mpp_train, mzp_train = calculate_mpp_mzp(X_train)
mpp_test, mzp_test = calculate_mpp_mzp(X_test)

X_train_mpp_mzp = np.concatenate([mpp_train, mzp_train], axis=-1)  # shape: (samples, features)
X_test_mpp_mzp = np.concatenate([mpp_test, mzp_test], axis=-1)

X_train_final = X_train_mpp_mzp[..., np.newaxis]  # shape: (samples, features, 1)
X_test_final = X_test_mpp_mzp[..., np.newaxis]
# X_train_final = X_train  # shape: (samples, features, 1)
# X_test_final = X_test


def build_model(hp):
    model = Sequential([
        Input(shape=(X_train_final.shape[1], 1)),
        
        
        Conv1D(filters=hp.Choice('conv1_filters', [16, 32, 64, 128]),
               kernel_size=hp.Choice('conv1_kernel', [2, 3, 5, 7]),
               padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=hp.Choice('conv2_filters', [32, 64, 128, 256]),
               kernel_size=hp.Choice('conv2_kernel', [2, 3, 5]),
               padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=hp.Choice('conv3_filters', [16, 32, 64, 128]),
               kernel_size=hp.Choice('conv3_kernel', [2, 3, 5]),
               padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        Activation('relu'),
        
        GlobalAveragePooling1D(),
        
    
        Dense(units=hp.Int('dense1_units', 64, 512, step=64),
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
        Dropout(hp.Float('dropout1', 0.3, 0.7, step=0.1)),
        
        Dense(units=hp.Int('dense2_units', 32, 256, step=32),
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
        Dropout(hp.Float('dropout2', 0.3, 0.7, step=0.1)),
        
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=hp.Float('lr', 1e-5, 1e-3, sampling='log')),
        metrics=['accuracy']
    )
    
    return model

tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='tuner_dir_db5_5g_new',
    project_name='cnn_gesture_classification'
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


tuner.search(X_train_final, y_train,
             epochs=30,
             batch_size=128,
             validation_data=(X_test_final, y_test),
             callbacks=[early_stop],
             verbose=2)


best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]


y_pred = np.argmax(best_model.predict(X_test_final), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Melhor acurácia CNN otimizada (DB2 - 5 gestos): {accuracy:.4f}")
print("Melhores hiperparâmetros encontrados:")
print(best_hp.values)

Trial 20 Complete [00h 00m 19s]
val_accuracy: 0.7192241549491882

Best val_accuracy So Far: 0.7345296144485474
Total elapsed time: 00h 08m 50s
  1/237 ━━━━━━━━━━━━━━━━━━━━ 25s 108ms/step

c:\Users\PC\Desktop\TCC_2025\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Melhor acurácia CNN otimizada (DB2 - 5 gestos): 0.7345
Melhores hiperparâmetros encontrados:
{'conv1_filters': 32, 'conv1_kernel': 5, 'conv2_filters': 256, 'conv2_kernel': 5, 'conv3_filters': 128, 'conv3_kernel': 3, 'dense1_units': 128, 'l2': 0.004739667552102512, 'dropout1': 0.4, 'dense2_units': 192, 'dropout2': 0.4, 'lr': 0.0001985090560549791}


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Activation, BatchNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, History
import keras_tuner as kt
import os
import pandas as pd
import matplotlib.pyplot as plt

def calculate_mpp_mzp(X):
    mpp, mzp = [], []
    for sample in X:
        mu0 = np.sqrt(np.sum(sample ** 2, axis=0))
        delta = np.diff(sample, axis=0)
        mu2 = np.sqrt(np.sum(delta ** 2, axis=0))
        delta2 = np.diff(delta, axis=0)
        mu4 = np.sqrt(np.sum(delta2 ** 2, axis=0))
        psi = np.nan_to_num(mu4 / mu2, nan=0.0, posinf=0.0, neginf=0.0)
        phi = np.nan_to_num(mu2 / mu0, nan=0.0, posinf=0.0, neginf=0.0)
        mpp.append(mu0 * psi)
        mzp.append(mu0 * phi)
    return np.stack(mpp), np.stack(mzp)

datasets = {
    "DB2_5G": r"D:\Stash\Datasets\db2_janelado\db2_5g_all_data.npz",
    "DB2_10G": r"D:\Stash\Datasets\db2_janelado\db2_10g_all_data.npz",
    "DB3_5G": r"D:\Stash\Datasets\db3_janelado\db3_5g_all_data.npz",
    "DB3_10G": r"D:\Stash\Datasets\db3_janelado\db3_10g_all_data.npz",
    "DB5_5G": r"D:\Stash\Datasets\db5_janelado\db5_5g_all_data.npz",
    "DB5_10G": r"D:\Stash\Datasets\db5_janelado\db5_10g_all_data.npz"
}

results = []

for db_name, data_path in datasets.items():
    print(f"\n🔹 Iniciando treinamento para {db_name}")

    data = np.load(data_path)
    X_train = data["X_train"]
    y_train = data["y_train"]
    X_test = data["X_test"]
    y_test = data["y_test"]

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)

    mpp_train, mzp_train = calculate_mpp_mzp(X_train)
    mpp_test, mzp_test = calculate_mpp_mzp(X_test)
    X_train_final = np.concatenate([mpp_train, mzp_train], axis=-1)[..., np.newaxis]
    X_test_final = np.concatenate([mpp_test, mzp_test], axis=-1)[..., np.newaxis]

    def build_model(hp):
        model = Sequential([
            Input(shape=(X_train_final.shape[1], 1)),
            Conv1D(filters=hp.Choice('conv1_filters', [16, 32, 64, 128]), kernel_size=hp.Choice('conv1_kernel', [2, 3, 5, 7]), padding='same', kernel_initializer='he_normal'),
            BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
            Conv1D(filters=hp.Choice('conv2_filters', [32, 64, 128, 256]), kernel_size=hp.Choice('conv2_kernel', [2, 3, 5]), padding='same', kernel_initializer='he_normal'),
            BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
            Conv1D(filters=hp.Choice('conv3_filters', [16, 32, 64, 128]), kernel_size=hp.Choice('conv3_kernel', [2, 3, 5]), padding='same', kernel_initializer='he_normal'),
            BatchNormalization(), Activation('relu'),
            GlobalAveragePooling1D(),
            Dense(units=hp.Int('dense1_units', 64, 512, step=64), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
            Dropout(hp.Float('dropout1', 0.3, 0.7, step=0.1)),
            Dense(units=hp.Int('dense2_units', 32, 256, step=32), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
            Dropout(hp.Float('dropout2', 0.3, 0.7, step=0.1)),
            Dense(len(np.unique(y_train)), activation='softmax')
        ])
        model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=hp.Float('lr', 1e-5, 1e-3, sampling='log')), metrics=['accuracy'])
        return model

    project_dir = os.path.join("CNN_tuner_dir", db_name)
    tuner = kt.BayesianOptimization(build_model, objective='val_accuracy', max_trials=50, directory=project_dir, project_name='cnn_gesture_classification')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history_callback = History()
    
    tuner.search(X_train_final, y_train, epochs=30, batch_size=128, validation_data=(X_test_final, y_test), callbacks=[early_stop, history_callback], verbose=1)

    best_model = tuner.get_best_models(1)[0]
    best_hp = tuner.get_best_hyperparameters(1)[0]
    y_pred = np.argmax(best_model.predict(X_test_final), axis=1)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"✅ Melhor acurácia para {db_name}: {accuracy:.4f}")
    print(f"Melhores hiperparâmetros para {db_name}: {best_hp.values}")


    results.append({
        'Dataset': db_name,
        'Accuracy': accuracy,
        **best_hp.values
    })

    if history_callback.history:
        plt.figure(figsize=(10, 4))
        plt.plot(history_callback.history['val_loss'], label='Validation Loss')
        plt.plot(history_callback.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Curvas de Validação para {db_name}')
        plt.xlabel('Epoch')
        plt.ylabel('Metric')
        plt.legend()
        plt.savefig(f"curva_{db_name}.png")
        plt.close()

results_df = pd.DataFrame(results)
results_df.to_csv("CNN_resultados_todosDBs.csv", index=False)
print("✅ Resultados salvos em CNN_resultados_todosDBs.csv")


Trial 50 Complete [00h 01m 22s]
val_accuracy: 0.6334514617919922

Best val_accuracy So Far: 0.651043713092804
Total elapsed time: 01h 13m 22s
  1/477 ━━━━━━━━━━━━━━━━━━━━ 1:06 140ms/step

c:\Users\PC\Desktop\TCC_2025\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
✅ Melhor acurácia para DB5_10G: 0.6510
Melhores hiperparâmetros para DB5_10G: {'conv1_filters': 64, 'conv1_kernel': 7, 'conv2_filters': 256, 'conv2_kernel': 3, 'conv3_filters': 128, 'conv3_kernel': 5, 'dense1_units': 384, 'l2': 0.0017738646500971463, 'dropout1': 0.4, 'dense2_units': 224, 'dropout2': 0.3, 'lr': 0.001}
✅ Resultados salvos em CNN_resultados_todosDBs.csv


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Activation, BatchNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import os

# === Função para calcular MPP e MZP ===
def calculate_mpp_mzp(X):
    mpp, mzp = [], []
    for sample in X:
        mu0 = np.sqrt(np.sum(sample ** 2, axis=0))
        delta = np.diff(sample, axis=0)
        mu2 = np.sqrt(np.sum(delta ** 2, axis=0))
        delta2 = np.diff(delta, axis=0)
        mu4 = np.sqrt(np.sum(delta2 ** 2, axis=0))
        psi = np.nan_to_num(mu4 / mu2)
        phi = np.nan_to_num(mu2 / mu0)
        mpp.append(mu0 * psi)
        mzp.append(mu0 * phi)
    return np.stack(mpp), np.stack(mzp)

# === Datasets e hiperparâmetros otimizados ===
config = [
    ("DB2_5G", r"D:\Stash\Datasets\db2_janelado\db2_5g_all_data.npz", {
        'conv1_filters':128,'conv1_kernel':2,'conv2_filters':256,'conv2_kernel':5,'conv3_filters':128,'conv3_kernel':5,
        'dense1_units':448,'l2':0.01,'dropout1':0.3,'dense2_units':32,'dropout2':0.6,'lr':0.001
    }),
    ("DB2_10G", r"D:\Stash\Datasets\db2_janelado\db2_10g_all_data.npz", {
        'conv1_filters':128,'conv1_kernel':7,'conv2_filters':256,'conv2_kernel':5,'conv3_filters':128,'conv3_kernel':5,
        'dense1_units':256,'l2':0.0060122,'dropout1':0.5,'dense2_units':160,'dropout2':0.3,'lr':0.001
    }),
    ("DB3_5G", r"D:\Stash\Datasets\db3_janelado\db3_5g_all_data.npz", {
        'conv1_filters':128,'conv1_kernel':7,'conv2_filters':256,'conv2_kernel':2,'conv3_filters':64,'conv3_kernel':3,
        'dense1_units':320,'l2':0.0004108,'dropout1':0.6,'dense2_units':64,'dropout2':0.3,'lr':0.001
    }),
    ("DB3_10G", r"D:\Stash\Datasets\db3_janelado\db3_10g_all_data.npz", {
        'conv1_filters':32,'conv1_kernel':3,'conv2_filters':128,'conv2_kernel':5,'conv3_filters':128,'conv3_kernel':5,
        'dense1_units':512,'l2':0.0008011,'dropout1':0.5,'dense2_units':160,'dropout2':0.4,'lr':0.0001133
    }),
    ("DB5_5G", r"D:\Stash\Datasets\db5_janelado\db5_5g_all_data.npz", {
        'conv1_filters':128,'conv1_kernel':5,'conv2_filters':256,'conv2_kernel':3,'conv3_filters':64,'conv3_kernel':5,
        'dense1_units':320,'l2':0.001925,'dropout1':0.4,'dense2_units':128,'dropout2':0.5,'lr':0.000196
    }),
    ("DB5_10G", r"D:\Stash\Datasets\db5_janelado\db5_10g_all_data.npz", {
        'conv1_filters':64,'conv1_kernel':7,'conv2_filters':256,'conv2_kernel':3,'conv3_filters':128,'conv3_kernel':5,
        'dense1_units':384,'l2':0.001773,'dropout1':0.4,'dense2_units':224,'dropout2':0.3,'lr':0.001
    })
]

# === Treinamento e salvamento ===
for name, data_path, hp in config:
    print(f"\n🔧 Treinando e salvando modelo para: {name}")


    data = np.load(data_path)
    X_train, y_train = data["X_train"], data["y_train"]
    X_test, y_test = data["X_test"], data["y_test"]

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)

    mpp_train, mzp_train = calculate_mpp_mzp(X_train)
    mpp_test, mzp_test = calculate_mpp_mzp(X_test)

    X_train_final = np.concatenate([mpp_train, mzp_train], axis=-1)[..., np.newaxis]
    X_test_final = np.concatenate([mpp_test, mzp_test], axis=-1)[..., np.newaxis]

    # Modelo com hiperparâmetros fixos
    model = Sequential([
        Input(shape=(X_train_final.shape[1], 1)),
        Conv1D(filters=hp['conv1_filters'], kernel_size=hp['conv1_kernel'], padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
        Conv1D(filters=hp['conv2_filters'], kernel_size=hp['conv2_kernel'], padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
        Conv1D(filters=hp['conv3_filters'], kernel_size=hp['conv3_kernel'], padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'),
        GlobalAveragePooling1D(),
        Dense(units=hp['dense1_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hp['l2'])),
        Dropout(hp['dropout1']),
        Dense(units=hp['dense2_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hp['l2'])),
        Dropout(hp['dropout2']),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=hp['lr']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])


    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train_final, y_train, epochs=50, batch_size=128, validation_data=(X_test_final, y_test), callbacks=[early_stop], verbose=2)


    model_name = f"modelo_CNN_otimizado_{name}.h5"
    model.save(model_name)
    print(f"✅ Modelo salvo como: {model_name}")



🔧 Treinando e salvando modelo para: DB2_5G
Epoch 1/50
560/560 - 12s - 22ms/step - accuracy: 0.4623 - loss: 1.5848 - val_accuracy: 0.4752 - val_loss: 1.4036
Epoch 2/50
560/560 - 10s - 18ms/step - accuracy: 0.6177 - loss: 1.0493 - val_accuracy: 0.6247 - val_loss: 1.0407
Epoch 3/50
560/560 - 12s - 21ms/step - accuracy: 0.6742 - loss: 0.9319 - val_accuracy: 0.6600 - val_loss: 0.9379
Epoch 4/50
560/560 - 11s - 20ms/step - accuracy: 0.7142 - loss: 0.8513 - val_accuracy: 0.6834 - val_loss: 0.9669
Epoch 5/50
560/560 - 10s - 18ms/step - accuracy: 0.7419 - loss: 0.7880 - val_accuracy: 0.6582 - val_loss: 1.0288
Epoch 6/50
560/560 - 10s - 18ms/step - accuracy: 0.7582 - loss: 0.7526 - val_accuracy: 0.7279 - val_loss: 0.7929
Epoch 7/50
560/560 - 10s - 18ms/step - accuracy: 0.7793 - loss: 0.6967 - val_accuracy: 0.6548 - val_loss: 1.0421
Epoch 8/50
560/560 - 10s - 18ms/step - accuracy: 0.7877 - loss: 0.6787 - val_accuracy: 0.7752 - val_loss: 0.7130
Epoch 9/50
560/560 - 10s - 18ms/step - accuracy: 0.8

✅ Modelo salvo como: modelo_CNN_otimizado_DB2_5G.h5

🔧 Treinando e salvando modelo para: DB2_10G
Epoch 1/50
1095/1095 - 22s - 20ms/step - accuracy: 0.4722 - loss: 1.7752 - val_accuracy: 0.4747 - val_loss: 1.7270
Epoch 2/50
1095/1095 - 20s - 18ms/step - accuracy: 0.6419 - loss: 1.1521 - val_accuracy: 0.6494 - val_loss: 1.1352
Epoch 3/50
1095/1095 - 20s - 19ms/step - accuracy: 0.7107 - loss: 0.9640 - val_accuracy: 0.6317 - val_loss: 1.2267
Epoch 4/50
1095/1095 - 20s - 18ms/step - accuracy: 0.7488 - loss: 0.8503 - val_accuracy: 0.6944 - val_loss: 1.0296
Epoch 5/50
1095/1095 - 20s - 18ms/step - accuracy: 0.7801 - loss: 0.7678 - val_accuracy: 0.6951 - val_loss: 0.9860
Epoch 6/50
1095/1095 - 19s - 18ms/step - accuracy: 0.7999 - loss: 0.7085 - val_accuracy: 0.7206 - val_loss: 0.9492
Epoch 7/50
1095/1095 - 20s - 18ms/step - accuracy: 0.8161 - loss: 0.6602 - val_accuracy: 0.7255 - val_loss: 0.9543
Epoch 8/50
1095/1095 - 19s - 18ms/step - accuracy: 0.8319 - loss: 0.6161 - val_accuracy: 0.7014 - 

✅ Modelo salvo como: modelo_CNN_otimizado_DB2_10G.h5

🔧 Treinando e salvando modelo para: DB3_5G


C:\Users\PC\AppData\Local\Temp\ipykernel_28724\3189033294.py:19: RuntimeWarning: invalid value encountered in divide
  psi = np.nan_to_num(mu4 / mu2)
C:\Users\PC\AppData\Local\Temp\ipykernel_28724\3189033294.py:20: RuntimeWarning: invalid value encountered in divide
  phi = np.nan_to_num(mu2 / mu0)


Epoch 1/50
148/148 - 4s - 28ms/step - accuracy: 0.4104 - loss: 1.4403 - val_accuracy: 0.4289 - val_loss: 1.4139
Epoch 2/50
148/148 - 2s - 13ms/step - accuracy: 0.6221 - loss: 1.0321 - val_accuracy: 0.5563 - val_loss: 1.1749
Epoch 3/50
148/148 - 2s - 13ms/step - accuracy: 0.6868 - loss: 0.8755 - val_accuracy: 0.5825 - val_loss: 1.2013
Epoch 4/50
148/148 - 2s - 13ms/step - accuracy: 0.7346 - loss: 0.7518 - val_accuracy: 0.5000 - val_loss: 1.5452
Epoch 5/50
148/148 - 2s - 13ms/step - accuracy: 0.7536 - loss: 0.7050 - val_accuracy: 0.5977 - val_loss: 1.2006
Epoch 6/50
148/148 - 2s - 13ms/step - accuracy: 0.7676 - loss: 0.6621 - val_accuracy: 0.6175 - val_loss: 1.1588
Epoch 7/50
148/148 - 2s - 13ms/step - accuracy: 0.7836 - loss: 0.6212 - val_accuracy: 0.5740 - val_loss: 1.5361
Epoch 8/50
148/148 - 2s - 13ms/step - accuracy: 0.7937 - loss: 0.5878 - val_accuracy: 0.6093 - val_loss: 1.2040
Epoch 9/50
148/148 - 2s - 13ms/step - accuracy: 0.8042 - loss: 0.5554 - val_accuracy: 0.5490 - val_loss:

...

C:\Users\PC\AppData\Local\Temp\ipykernel_28724\3189033294.py:19: RuntimeWarning: invalid value encountered in divide
  psi = np.nan_to_num(mu4 / mu2)
C:\Users\PC\AppData\Local\Temp\ipykernel_28724\3189033294.py:20: RuntimeWarning: invalid value encountered in divide
  phi = np.nan_to_num(mu2 / mu0)


Epoch 1/50
304/304 - 6s - 19ms/step - accuracy: 0.1747 - loss: 2.5564 - val_accuracy: 0.2458 - val_loss: 2.4345
Epoch 2/50
304/304 - 3s - 10ms/step - accuracy: 0.2580 - loss: 2.3398 - val_accuracy: 0.3017 - val_loss: 2.2138
Epoch 3/50
304/304 - 3s - 10ms/step - accuracy: 0.3316 - loss: 2.1236 - val_accuracy: 0.3692 - val_loss: 2.0472
Epoch 4/50
304/304 - 3s - 10ms/step - accuracy: 0.3926 - loss: 1.9539 - val_accuracy: 0.4054 - val_loss: 1.9464
Epoch 5/50
304/304 - 3s - 10ms/step - accuracy: 0.4387 - loss: 1.8231 - val_accuracy: 0.4121 - val_loss: 1.8959
Epoch 6/50
304/304 - 3s - 10ms/step - accuracy: 0.4723 - loss: 1.7233 - val_accuracy: 0.4394 - val_loss: 1.8285
Epoch 7/50
304/304 - 3s - 10ms/step - accuracy: 0.4959 - loss: 1.6469 - val_accuracy: 0.4268 - val_loss: 1.8568
Epoch 8/50
304/304 - 3s - 10ms/step - accuracy: 0.5174 - loss: 1.5801 - val_accuracy: 0.4515 - val_loss: 1.8017
Epoch 9/50
304/304 - 3s - 10ms/step - accuracy: 0.5409 - loss: 1.5210 - val_accuracy: 0.4362 - val_loss:

✅ Modelo salvo como: modelo_CNN_otimizado_DB3_10G.h5

🔧 Treinando e salvando modelo para: DB5_5G
Epoch 1/50
123/123 - 4s - 35ms/step - accuracy: 0.4324 - loss: 1.8732 - val_accuracy: 0.5779 - val_loss: 1.6270
Epoch 2/50
123/123 - 2s - 18ms/step - accuracy: 0.6300 - loss: 1.4135 - val_accuracy: 0.6548 - val_loss: 1.2999
Epoch 3/50
123/123 - 2s - 17ms/step - accuracy: 0.6953 - loss: 1.2178 - val_accuracy: 0.6688 - val_loss: 1.2197
Epoch 4/50
123/123 - 2s - 17ms/step - accuracy: 0.7374 - loss: 1.0856 - val_accuracy: 0.6682 - val_loss: 1.2291
Epoch 5/50
123/123 - 2s - 17ms/step - accuracy: 0.7634 - loss: 0.9899 - val_accuracy: 0.7110 - val_loss: 1.0858
Epoch 6/50
123/123 - 2s - 17ms/step - accuracy: 0.7838 - loss: 0.9131 - val_accuracy: 0.7125 - val_loss: 1.0785
Epoch 7/50
123/123 - 2s - 18ms/step - accuracy: 0.8056 - loss: 0.8407 - val_accuracy: 0.6697 - val_loss: 1.1968
Epoch 8/50
123/123 - 2s - 18ms/step - accuracy: 0.8190 - loss: 0.7826 - val_accuracy: 0.7249 - val_loss: 1.0331
Epoch 9

✅ Modelo salvo como: modelo_CNN_otimizado_DB5_5G.h5

🔧 Treinando e salvando modelo para: DB5_10G
Epoch 1/50
241/241 - 6s - 26ms/step - accuracy: 0.5092 - loss: 1.8994 - val_accuracy: 0.5180 - val_loss: 1.7415
Epoch 2/50
241/241 - 4s - 16ms/step - accuracy: 0.6785 - loss: 1.2202 - val_accuracy: 0.5528 - val_loss: 1.5433
Epoch 3/50
241/241 - 4s - 16ms/step - accuracy: 0.7358 - loss: 0.9806 - val_accuracy: 0.5830 - val_loss: 1.4672
Epoch 4/50
241/241 - 4s - 16ms/step - accuracy: 0.7747 - loss: 0.8326 - val_accuracy: 0.6376 - val_loss: 1.3097
Epoch 5/50
241/241 - 4s - 17ms/step - accuracy: 0.8013 - loss: 0.7298 - val_accuracy: 0.5887 - val_loss: 1.5475
Epoch 6/50
241/241 - 4s - 17ms/step - accuracy: 0.8277 - loss: 0.6527 - val_accuracy: 0.6364 - val_loss: 1.3118
Epoch 7/50
241/241 - 4s - 17ms/step - accuracy: 0.8501 - loss: 0.5851 - val_accuracy: 0.5828 - val_loss: 1.6125
Epoch 8/50
241/241 - 4s - 17ms/step - accuracy: 0.8676 - loss: 0.5289 - val_accuracy: 0.5963 - val_loss: 1.6197
Epoch 9

✅ Modelo salvo como: modelo_CNN_otimizado_DB5_10G.h5


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Activation, BatchNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, History
import keras_tuner as kt
import os
import pandas as pd
import matplotlib.pyplot as plt

def calculate_mpp_mzp(X):
    mpp, mzp = [], []
    for sample in X:
        mu0 = np.sqrt(np.sum(sample ** 2, axis=0))
        delta = np.diff(sample, axis=0)
        mu2 = np.sqrt(np.sum(delta ** 2, axis=0))
        delta2 = np.diff(delta, axis=0)
        mu4 = np.sqrt(np.sum(delta2 ** 2, axis=0))
        psi = np.nan_to_num(mu4 / mu2, nan=0.0, posinf=0.0, neginf=0.0)
        phi = np.nan_to_num(mu2 / mu0, nan=0.0, posinf=0.0, neginf=0.0)
        mpp.append(mu0 * psi)
        mzp.append(mu0 * phi)
    return np.stack(mpp), np.stack(mzp)

datasets = {
    "DB2_5G": r"D:\Stash\Datasets\db2_janelado\db2_5g_all_data.npz",
    "DB2_10G": r"D:\Stash\Datasets\db2_janelado\db2_10g_all_data.npz",
    "DB3_5G": r"D:\Stash\Datasets\db3_janelado\db3_5g_all_data.npz",
    "DB3_10G": r"D:\Stash\Datasets\db3_janelado\db3_10g_all_data.npz",
    "DB5_5G": r"D:\Stash\Datasets\db5_janelado\db5_5g_all_data.npz",
    "DB5_10G": r"D:\Stash\Datasets\db5_janelado\db5_10g_all_data.npz"
}


baseline_results = []

for db_name, data_path in datasets.items():
    print(f"\n🔸 Treinamento BASELINE para {db_name}")

    data = np.load(data_path)
    X_train = data["X_train"]
    y_train = data["y_train"]
    X_test = data["X_test"]
    y_test = data["y_test"]

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)

    mpp_train, mzp_train = calculate_mpp_mzp(X_train)
    mpp_test, mzp_test = calculate_mpp_mzp(X_test)
    X_train_final = np.concatenate([mpp_train, mzp_train], axis=-1)[..., np.newaxis]
    X_test_final = np.concatenate([mpp_test, mzp_test], axis=-1)[..., np.newaxis]

    model = Sequential([
        Input(shape=(X_train_final.shape[1], 1)),
        Conv1D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'),
        GlobalAveragePooling1D(),
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
        Dropout(0.5),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])


    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train_final, y_train, epochs=30, batch_size=128, validation_data=(X_test_final, y_test), callbacks=[early_stop], verbose=1)


    y_pred = np.argmax(model.predict(X_test_final), axis=1)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"✅ Acurácia BASELINE para {db_name}: {accuracy:.4f}")

    baseline_results.append({
        'Dataset': db_name,
        'Accuracy': accuracy,
        'conv1_filters': 64,
        'conv1_kernel': 3,
        'conv2_filters': 128,
        'conv2_kernel': 3,
        'conv3_filters': 64,
        'conv3_kernel': 3,
        'dense1_units': 256,
        'dense2_units': 128,
        'dropout1': 0.5,
        'dropout2': 0.5,
        'l2': 1e-3,
        'lr': 1e-4
    })

    plt.figure(figsize=(10, 4))
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Curvas BASELINE para {db_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Metric')
    plt.legend()
    plt.savefig(f"curva_baseline_{db_name}.png")
    plt.close()

baseline_df = pd.DataFrame(baseline_results)
baseline_df.to_csv("CNN_baseline_todosDBs.csv", index=False)
print("✅ Resultados BASELINE salvos em CNN_baseline_todosDBs.csv")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Activation, BatchNormalization, Input, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import os
import pandas as pd
import matplotlib.pyplot as plt

def calculate_mpp_mzp(X):
    mpp, mzp = [], []
    for sample in X:
        mu0 = np.sqrt(np.sum(sample ** 2, axis=0))
        delta = np.diff(sample, axis=0)
        mu2 = np.sqrt(np.sum(delta ** 2, axis=0))
        delta2 = np.diff(delta, axis=0)
        mu4 = np.sqrt(np.sum(delta2 ** 2, axis=0))
        psi = np.nan_to_num(mu4 / mu2, nan=0.0, posinf=0.0, neginf=0.0)
        phi = np.nan_to_num(mu2 / mu0, nan=0.0, posinf=0.0, neginf=0.0)
        mpp.append(mu0 * psi)
        mzp.append(mu0 * phi)
    return np.stack(mpp), np.stack(mzp)

datasets = {
    "DB2_5G": r"D:\Stash\Datasets\db2_janelado\db2_5g_all_data.npz",
    "DB2_10G": r"D:\Stash\Datasets\db2_janelado\db2_10g_all_data.npz",
    "DB3_5G": r"D:\Stash\Datasets\db3_janelado\db3_5g_all_data.npz",
    "DB3_10G": r"D:\Stash\Datasets\db3_janelado\db3_10g_all_data.npz",
    "DB5_5G": r"D:\Stash\Datasets\db5_janelado\db5_5g_all_data.npz",
    "DB5_10G": r"D:\Stash\Datasets\db5_janelado\db5_10g_all_data.npz"
}

hybrid_results = []

for db_name, data_path in datasets.items():
    print(f"\n🔸 Treinamento HÍBRIDO CNN+LSTM para {db_name}")

    data = np.load(data_path)
    X_train = data["X_train"]
    y_train = data["y_train"]
    X_test = data["X_test"]
    y_test = data["y_test"]

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)

    mpp_train, mzp_train = calculate_mpp_mzp(X_train)
    mpp_test, mzp_test = calculate_mpp_mzp(X_test)
    X_train_final = np.concatenate([mpp_train, mzp_train], axis=-1)[..., np.newaxis]
    X_test_final = np.concatenate([mpp_test, mzp_test], axis=-1)[..., np.newaxis]

    # Modelo HÍBRIDO CNN + LSTM
    model = Sequential([
        Input(shape=(X_train_final.shape[1], 1)),
        Conv1D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(), Activation('relu'),
        Reshape((-1, 64)),
        LSTM(64, return_sequences=False),
        Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
        Dropout(0.5),
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])


    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    history = model.fit(X_train_final, y_train, epochs=30, batch_size=128, validation_data=(X_test_final, y_test), callbacks=[early_stop], verbose=1)


    y_pred = np.argmax(model.predict(X_test_final), axis=1)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"✅ Acurácia HÍBRIDA para {db_name}: {accuracy:.4f}")

    hybrid_results.append({
        'Dataset': db_name,
        'Accuracy': accuracy,
        'conv1_filters': 64,
        'conv1_kernel': 3,
        'conv2_filters': 128,
        'conv2_kernel': 3,
        'conv3_filters': 64,
        'conv3_kernel': 3,
        'lstm_units': 64,
        'dense1_units': 256,
        'dense2_units': 128,
        'dropout1': 0.5,
        'dropout2': 0.5,
        'l2': 1e-3,
        'lr': 1e-4
    })

    plt.figure(figsize=(10, 4))
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Curvas HÍBRIDAS para {db_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Metric')
    plt.legend()
    plt.savefig(f"curva_hibrido_{db_name}.png")
    plt.close()

hybrid_df = pd.DataFrame(hybrid_results)
hybrid_df.to_csv("CNN_LSTM_hibrido_todosDBs.csv", index=False)
print("✅ Resultados HÍBRIDOS salvos em CNN_LSTM_hibrido_todosDBs.csv")


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Activation, BatchNormalization, Input, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os
import pandas as pd
import matplotlib.pyplot as plt

def calculate_mpp_mzp(X):
    mpp, mzp = [], []
    for sample in X:
        mu0 = np.sqrt(np.sum(sample ** 2, axis=0))
        delta = np.diff(sample, axis=0)
        mu2 = np.sqrt(np.sum(delta ** 2, axis=0))
        delta2 = np.diff(delta, axis=0)
        mu4 = np.sqrt(np.sum(delta2 ** 2, axis=0))
        psi = np.nan_to_num(mu4 / mu2, nan=0.0, posinf=0.0, neginf=0.0)
        phi = np.nan_to_num(mu2 / mu0, nan=0.0, posinf=0.0, neginf=0.0)
        mpp.append(mu0 * psi)
        mzp.append(mu0 * phi)
    return np.stack(mpp), np.stack(mzp)

datasets = {
    "DB2_5G": r"D:\Stash\Datasets\db2_janelado\db2_5g_all_data.npz",
    "DB2_10G": r"D:\Stash\Datasets\db2_janelado\db2_10g_all_data.npz",
    "DB3_5G": r"D:\Stash\Datasets\db3_janelado\db3_5g_all_data.npz",
    "DB3_10G": r"D:\Stash\Datasets\db3_janelado\db3_10g_all_data.npz",
    "DB5_5G": r"D:\Stash\Datasets\db5_janelado\db5_5g_all_data.npz",
    "DB5_10G": r"D:\Stash\Datasets\db5_janelado\db5_10g_all_data.npz"
}

results = []

for db_name, data_path in datasets.items():
    print(f"\n🔹 Iniciando otimização BAYESIANA CNN+LSTM para {db_name}")

    data = np.load(data_path)
    X_train = data["X_train"]
    y_train = data["y_train"]
    X_test = data["X_test"]
    y_test = data["y_test"]

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.transform(y_test)

    mpp_train, mzp_train = calculate_mpp_mzp(X_train)
    mpp_test, mzp_test = calculate_mpp_mzp(X_test)
    X_train_final = np.concatenate([mpp_train, mzp_train], axis=-1)[..., np.newaxis]
    X_test_final = np.concatenate([mpp_test, mzp_test], axis=-1)[..., np.newaxis]

    def build_model(hp):
        model = Sequential([
            Input(shape=(X_train_final.shape[1], 1)),
            Conv1D(filters=hp.Choice('conv1_filters', [16, 32, 64, 128]), kernel_size=hp.Choice('conv1_kernel', [2, 3, 5, 7]), padding='same', kernel_initializer='he_normal'),
            BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
            Conv1D(filters=hp.Choice('conv2_filters', [32, 64, 128, 256]), kernel_size=hp.Choice('conv2_kernel', [2, 3, 5]), padding='same', kernel_initializer='he_normal'),
            BatchNormalization(), Activation('relu'), MaxPooling1D(pool_size=2),
            Conv1D(filters=hp.Choice('conv3_filters', [16, 32, 64, 128]), kernel_size=hp.Choice('conv3_kernel', [2, 3, 5]), padding='same', kernel_initializer='he_normal'),
            BatchNormalization(), Activation('relu'),
            Reshape((-1, hp.Choice('reshape_features', [16, 32, 64, 128]))),
            LSTM(hp.Int('lstm_units', 32, 128, step=32), return_sequences=False),
            Dense(hp.Int('dense1_units', 64, 512, step=64), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
            Dropout(hp.Float('dropout1', 0.3, 0.7, step=0.1)),
            Dense(hp.Int('dense2_units', 32, 256, step=32), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
            Dropout(hp.Float('dropout2', 0.3, 0.7, step=0.1)),
            Dense(len(np.unique(y_train)), activation='softmax')
        ])
        model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=hp.Float('lr', 1e-5, 1e-3, sampling='log')), metrics=['accuracy'])
        return model

    project_dir = os.path.join("CNN_LSTM_tuner_dir", db_name)
    tuner = kt.BayesianOptimization(build_model, objective='val_accuracy', max_trials=30, directory=project_dir, project_name='cnn_lstm_gesture_classification')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    tuner.search(X_train_final, y_train, epochs=30, batch_size=128, validation_data=(X_test_final, y_test), callbacks=[early_stop], verbose=1)

    best_model = tuner.get_best_models(1)[0]
    best_hp = tuner.get_best_hyperparameters(1)[0]
    y_pred = np.argmax(best_model.predict(X_test_final), axis=1)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"✅ Melhor acurácia HÍBRIDA (otimizada) para {db_name}: {accuracy:.4f}")
    print(f"Melhores hiperparâmetros para {db_name}: {best_hp.values}")

    results.append({'Dataset': db_name, 'Accuracy': accuracy, **best_hp.values})

    model_filename = f"modelo_hibrido_otimizado_{db_name}.h5"
    best_model.save(model_filename)
    print(f"✅ Modelo salvo como {model_filename}")


    plt.figure(figsize=(10, 4))
    plt.title(f'Curvas HÍBRIDAS OTIMIZADAS para {db_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Metric')
    plt.legend()
    plt.savefig(f"curva_hibrido_otimizado_{db_name}.png")
    plt.close()

results_df = pd.DataFrame(results)
results_df.to_csv("CNN_LSTM_hibrido_otimizado_todosDBs.csv", index=False)
print("✅ Resultados HÍBRIDOS OTIMIZADOS salvos em CNN_LSTM_hibrido_otimizado_todosDBs.csv")


Trial 30 Complete [00h 02m 24s]
val_accuracy: 0.6018773913383484

Best val_accuracy So Far: 0.6521596312522888
Total elapsed time: 01h 06m 30s


c:\Users\PC\Desktop\TCC_2025\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 44 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


477/477 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
✅ Melhor acurácia HÍBRIDA (otimizada) para DB5_10G: 0.6522


Melhores hiperparâmetros para DB5_10G: {'conv1_filters': 16, 'conv1_kernel': 7, 'conv2_filters': 128, 'conv2_kernel': 5, 'conv3_filters': 64, 'conv3_kernel': 5, 'reshape_features': 128, 'lstm_units': 128, 'dense1_units': 128, 'l2': 0.00015996109792129216, 'dropout1': 0.5, 'dense2_units': 32, 'dropout2': 0.3, 'lr': 0.0007765767620141046}
✅ Modelo salvo como modelo_hibrido_otimizado_DB5_10G.h5
✅ Resultados HÍBRIDOS OTIMIZADOS salvos em CNN_LSTM_hibrido_otimizado_todosDBs.csv


C:\Users\PC\AppData\Local\Temp\ipykernel_14728\803235727.py:103: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
import keras_tuner as kt

data_path = r"D:\Stash\Datasets\db3_janelado\db3_5g_all_data.npz"
data = np.load(data_path)
X_train = data["X_train"]
y_train = data["y_train"]
X_test = data["X_test"]
y_test = data["y_test"]


encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

print("Formato final dos dados:", X_train.shape)
input_shape = (500, 12)   
num_classes = len(np.unique(y_train))

def build_model(hp):
    model = keras.Sequential(name="EMG_Gesture_CNN")
    model.add(layers.Conv1D(
        filters=hp.Int('conv1_filters', 32, 128, step=32),
        kernel_size=hp.Choice('conv1_kernel', [3, 5, 7]), 
        activation='relu',
        padding='same',
        input_shape=input_shape
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(hp.Float('conv1_dropout', 0.0, 0.5, step=0.1)))

    conv_blocks = hp.Int('conv_blocks_additional', 0, 2)
    for i in range(conv_blocks):
        filters = hp.Int(f'conv{i+2}_filters', 32, 128, step=32)
        kernel_size = hp.Choice(f'conv{i+2}_kernel', [3, 5, 7])
        model.add(layers.Conv1D(filters=filters, kernel_size=kernel_size,
                                activation='relu', padding='same'))
        model.add(layers.BatchNormalization())
        pool_type = hp.Choice(f'pooling{i+2}', ['max', 'avg'])
        if pool_type == 'max':
            model.add(layers.MaxPooling1D(pool_size=2))
        else:
            model.add(layers.AveragePooling1D(pool_size=2))
        model.add(layers.Dropout(hp.Float(f'conv{i+2}_dropout', 0.0, 0.5, step=0.1)))

    model.add(layers.GlobalAveragePooling1D())
    dense_units = hp.Int('dense_units', 0, 128, step=32)
    if dense_units > 0:
        model.add(layers.Dense(dense_units, activation='relu'))
        model.add(layers.Dropout(hp.Float('dense_dropout', 0.0, 0.5, step=0.1)))

    model.add(layers.Dense(num_classes, activation='softmax'))
    learning_rate = hp.Choice('learning_rate', [1e-3, 5e-4, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model


tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    directory='my_tuner_dir',
    project_name='sEMG_gesture_CNN_tuning'
)


tuner.search(X_train, y_train, 
             epochs=30, 
             validation_data=(X_test, y_test),
             batch_size=64,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)])


best_model = tuner.get_best_models(1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Melhores hiperparâmetros encontrados:", best_hp.values)
best_model.summary()


loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Acurácia no conjunto de teste: {accuracy:.2%}")

y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_test  
print(classification_report(y_true, y_pred_classes))
print("F1-score (macro):", f1_score(y_true, y_pred_classes, average='macro'))


Trial 20 Complete [00h 02m 17s]
val_accuracy: 0.5043230652809143

Best val_accuracy So Far: 0.652572751045227
Total elapsed time: 04h 10m 39s
Melhores hiperparâmetros encontrados: {'conv1_filters': 96, 'conv1_kernel': 3, 'conv1_dropout': 0.4, 'conv_blocks_additional': 0, 'dense_units': 128, 'learning_rate': 0.0005, 'conv2_filters': 96, 'conv2_kernel': 5, 'pooling2': 'avg', 'conv2_dropout': 0.2, 'conv3_filters': 64, 'conv3_kernel': 5, 'pooling3': 'avg', 'conv3_dropout': 0.0, 'dense_dropout': 0.30000000000000004}


c:\Users\PC\Desktop\TCC_2025\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "EMG_Gesture_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 500, 96)        │         3,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 500, 96)        │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 250, 96)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 250, 96)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 96)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,997 (66.39 KB)

 Trainable params: 16,805 (65.64 KB)

 Non-trainable params: 192 (768.00 B)

297/297 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6861 - loss: 0.8994
Acurácia no conjunto de teste: 65.26%
297/297 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
              precision    recall  f1-score   support

           0       0.67      0.69      0.68      1837
           1       0.64      0.69      0.67      1857
           2       0.64      0.56      0.60      1896
           3       0.62      0.70      0.66      2009
           4       0.70      0.61      0.66      1885

    accuracy                           0.65      9484
   macro avg       0.66      0.65      0.65      9484
weighted avg       0.65      0.65      0.65      9484

F1-score (macro): 0.6518356802698515


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, auc
from tensorflow.keras.callbacks import EarlyStopping


test_path = r"D:\Stash\Datasets\db3_features_all\5g\features_test.npz"
train_path =  r"D:\Stash\Datasets\db3_features_all\5g\features_train.npz"

teste = np.load(test_path)
treino = np.load(train_path)
X_train = treino["X"]
y_train = treino["y"]
X_test = teste["X"]
y_test = teste["y"]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


def build_model(hp):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(units=hp.Int('dense1_units', 128, 512, step=128),
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
        BatchNormalization(),
        Dropout(hp.Float('dropout1', 0.3, 0.7, step=0.1)),
        
        Dense(units=hp.Int('dense2_units', 64, 256, step=64),
              activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2', 1e-4, 1e-2, sampling='log'))),
        BatchNormalization(),
        Dropout(hp.Float('dropout2', 0.3, 0.7, step=0.1)),
        
        Dense(len(np.unique(y_train)), activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('lr', 1e-4, 1e-3, sampling='log')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='tuner_dir_db3_5g_9f',
    project_name='dnn_features_classification'
)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


tuner.search(X_train, y_train,
             epochs=5,
             batch_size=256,
             validation_data=(X_test, y_test),
             callbacks=[early_stop],
             verbose=2)

best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]


output_dir = "resultados_db3_5g"
os.makedirs(output_dir, exist_ok=True)


with open(os.path.join(output_dir, "best_hyperparameters.txt"), "w") as f:
    for param, value in best_hp.values.items():
        f.write(f"{param}: {value}\n")


best_model.save(os.path.join(output_dir, "best_model.h5"))


y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


accuracy = accuracy_score(y_test, y_pred)
print(f"Melhor acurácia DNN otimizada (DB2 - 5g): {accuracy:.4f}")
with open(os.path.join(output_dir, "final_accuracy.txt"), "w") as f:
    f.write(f"Final accuracy: {accuracy:.4f}\n")


cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_train))
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title("Confusion Matrix")
plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))

report = classification_report(y_test, y_pred)
with open(os.path.join(output_dir, "classification_report.txt"), "w") as f:
    f.write(report)


(18911, 144)
(18911,)
(9484, 144)
(9484,)


NameError: name 'tf' is not defined